#### Imports

#### Collecte de données
afficher nombre de colonnes et de lignes


## Pré-traitement des données

#### Sélection des colonnes les plus pertinentes à évaluer

#### Supprimer les doublons

#### Conversion des types de colonnes
- Conversion de Texte en Chiffres
- Extraction de Nombres
- Uniformisation des Données

## Analyse descriptive des données

#### Relation entre la note globale et les prix des locations à Paris

#### Prix moyen par type de propriété à Paris

#### Prix en fonction de Location disponible à Paris

#### Répartition des locations disponible et non disponible pour Paris

## Gestion des valeurs abberantes

## Séparation des données en ensemble d'entraînement et de test

#### Régression linéaire simple

#### Régression linéaire multiple

#### Decision tree